In [1]:
from parlai.core.params import ParlaiParser
from parlai.core.dict import DictionaryAgent
from parlai.scripts.train_model import setup_args as tm_setupargs
from parlai.scripts.train_model import TrainLoop
from parlai.core.agents import create_agent
from parlai.core.worlds import create_task
from itertools import islice
import json
import time
import os

In [10]:
def get_responses(agent, text, topic):
    # topic = classify(text)
    obs = {"text": text, 'episode_done': False, 'topic': topic}
    agent.observe(obs)
    batch = agent.batchify([agent.observation])
    batchsize = (
        batch.text_vec.size(0)
        if batch.text_vec is not None
        else batch.image.size(0)
    )
    agent.model.eval()
    cands = agent.fixed_candidates[topic]
    cand_vecs = agent.fixed_candidate_vecs[topic]
    cand_encs = agent.fixed_candidate_encs[topic]
    scores = agent.score_candidates(batch, cand_vecs, cand_encs=cand_encs)

    if agent.rank_top_k > 0:
        sorted_scores, ranks = scores.topk(
            min(agent.rank_top_k, scores.size(1)), 1, largest=True
        )
    else:
        sorted_scores, ranks = scores.sort(1, descending=True)

    # sorted_scores = F.softmax(sorted_scores, dim=-1)

    ranks = ranks.cpu()
    max_preds = agent.opt['cap_num_predictions']
    cand_preds = []
    for i, ordering in enumerate(ranks):
        if cand_vecs.dim() == 2:
            cand_list = cands
        elif cand_vecs.dim() == 3:
            cand_list = cands[i]
        # using a generator instead of a list comprehension allows
        # to cap the number of elements.
        cand_preds_generator = (
            cand_list[rank] for rank in ordering if rank < len(cand_list)
        )
        cand_preds.append(list(islice(cand_preds_generator, max_preds)))

    if (
        agent.opt.get('repeat_blocking_heuristic', True)
        and agent.eval_candidates == 'fixed'
    ):
        cand_preds = agent.block_repeats(cand_preds)

    preds = [cand_preds[i][0] for i in range(batchsize)]
    return preds, cand_preds

In [2]:
SHARED = {}
def setup_interactive(shared):
    parser = tm_setupargs()
    parser.set_params(batchsize=5)
    SHARED['opt'] = parser.parse_args([], print_args=True)

    SHARED['opt']['task'] = 'wizard_of_wikipedia:generator:random_split'
    SHARED['opt']['model'] = 'projects.wizard_of_wikipedia.generator.agents:EndToEndAgent'
    SHARED['opt']['dict_tokenizer'] = 'bpe'
    SHARED['opt']['dict_lower'] = True
    SHARED['opt']['dict_file'] = '/tmp/end2end_generator/model.dict'
    SHARED['opt']['model_file'] = '/tmp/end2end_generator/model'
    SHARED['opt']['no_cuda'] = True
    SHARED['opt']['truncate'] = 128
    SHARED['opt']['interactive_mode'] = True
    SHARED['opt']['embedding_size'] = 256
    SHARED['opt']['n_heads'] = 2
    SHARED['opt']['n_layers'] = 5
    SHARED['opt']['ffn_size'] = 512
    SHARED['opt']['dropout'] = 0.2
    SHARED['opt']['n_positions']= 128
    SHARED['opt']['max_knowledge']= 32
    SHARED['opt']['knowledge_alpha']= 0.95
    SHARED['opt']['knowledge_truncate']= 32
    SHARED['opt']['learningrate']= 5e-4
    SHARED['opt']['warmup_updates']= 5000
    SHARED['opt']['clip']= 0.1
    SHARED['opt']['lr_scheduler']= 'invsqrt'
    SHARED['opt']['embedding_type']= 'fasttext'
    SHARED['opt']['beam_size']= 1
    SHARED['opt']['skip_generation']= False
    
    SHARED['opt']['optimizer']= 'adam'
    SHARED['opt']['momentum']= 0
    SHARED['opt']['warmup_rate']= 1e-4
    SHARED['opt']['history_size']= -1
    SHARED['opt']['rank_candidates']= False
    SHARED['opt']['attention_dropout']= 0.0
    SHARED['opt']['relu_dropout']= 0.0
    SHARED['opt']['learn_positional_embeddings']= True
    SHARED['opt']['embeddings_scale']= True
    SHARED['opt']['activation']= 'gelu'
    SHARED['opt']['variant']= 'aiayn'
    SHARED['opt']['output_scaling']= 1.0
    # Create model and assign it to the specified task
    SHARED['agent'] = create_agent(SHARED.get('opt'), requireModelExists=False)
    SHARED['world'] = create_task(SHARED.get('opt'), [SHARED['agent']])

In [3]:
setup_interactive(SHARED)

[ Main ParlAI Arguments: ] 
[  batchsize: 5 ]
[  datapath: d:\research\parlai_seq2seq\data ]
[  datatype: train ]
[  download_path: d:\research\parlai_seq2seq\downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  init_opt: None ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: None ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: None ]
[  model_file: None ]
[ Training Loop Arguments: ] 
[  aggregate_micro: False ]
[  display_examples: False ]
[  eval_batchsize: None ]
[  evaltask: None ]
[  load_from_checkpoint: False ]
[  max_train_time: -1 ]
[  metrics: default ]
[  num_epochs: -1 ]
[  save_after_valid: False ]
[  save_every_n_secs: -1 ]
[  short_final_eval: False ]
[  validation_cutoff: 1.0 ]
[  validation_every_n_epochs: -1 ]
[  validation_every_n_secs: -1 ]
[  validation_max_exs: -1 ]
[  validation_metric: accuracy ]
[  validation_metric_mode: None ]
[  validation_patience: 10 ]
[  validation_share_a

d:\research\parlai_seq2seq\parlai\agents\transformer\modules.py:33: UserWarning: Installing APEX can give a significant speed boost.
  warn_once("Installing APEX can give a significant speed boost.")


[EndToEnd: full interactive mode on.]
Initialized embeddings for 1 tokens (25.0%) from fasttext.
Total parameters: 6656000
Trainable parameters:  6656000
[creating task(s): wizard_of_wikipedia:generator:random_split]
loading: d:\research\parlai_seq2seq\data\wizard_of_wikipedia\train.json


In [9]:
SHARED['world'].__dict__

{'id': 'wizard_of_wikipedia',
 'opt': {'init_opt': None,
  'show_advanced_args': False,
  'task': 'wizard_of_wikipedia',
  'download_path': 'd:\\research\\parlai_seq2seq\\downloads',
  'datatype': 'train',
  'image_mode': 'raw',
  'numthreads': 1,
  'hide_labels': False,
  'multitask_weights': [1],
  'batchsize': 5,
  'datapath': 'd:\\research\\parlai_seq2seq\\data',
  'model': 'projects.wizard_of_wikipedia.generator.agents:EndToEndAgent',
  'model_file': '/tmp/end2end_generator/model',
  'init_model': None,
  'dict_class': None,
  'evaltask': None,
  'eval_batchsize': None,
  'display_examples': False,
  'num_epochs': -1,
  'max_train_time': -1,
  'validation_every_n_secs': -1,
  'save_every_n_secs': -1,
  'save_after_valid': False,
  'validation_every_n_epochs': -1,
  'validation_max_exs': -1,
  'short_final_eval': False,
  'validation_patience': 10,
  'validation_metric': 'accuracy',
  'validation_metric_mode': None,
  'validation_cutoff': 1.0,
  'load_from_checkpoint': False,
  'va

In [11]:
SHARED['world'].worlds[0].__dict__

{'id': 'WizTeacher',
 'opt': {'init_opt': None,
  'show_advanced_args': False,
  'task': 'wizard_of_wikipedia',
  'download_path': 'd:\\research\\parlai_seq2seq\\downloads',
  'datatype': 'train',
  'image_mode': 'raw',
  'numthreads': 1,
  'hide_labels': False,
  'multitask_weights': [1],
  'batchsize': 5,
  'datapath': 'd:\\research\\parlai_seq2seq\\data',
  'model': 'projects.wizard_of_wikipedia.generator.agents:EndToEndAgent',
  'model_file': '/tmp/end2end_generator/model',
  'init_model': None,
  'dict_class': None,
  'evaltask': None,
  'eval_batchsize': None,
  'display_examples': False,
  'num_epochs': -1,
  'max_train_time': -1,
  'validation_every_n_secs': -1,
  'save_every_n_secs': -1,
  'save_after_valid': False,
  'validation_every_n_epochs': -1,
  'validation_max_exs': -1,
  'short_final_eval': False,
  'validation_patience': 10,
  'validation_metric': 'accuracy',
  'validation_metric_mode': None,
  'validation_cutoff': 1.0,
  'load_from_checkpoint': False,
  'validation_

In [21]:
SHARED['agent'].dict.__dict__

{'opt': {'init_opt': None,
  'show_advanced_args': False,
  'task': 'wizard_of_wikipedia:generator:random_split',
  'download_path': 'd:\\research\\parlai_seq2seq\\downloads',
  'datatype': 'train',
  'image_mode': 'raw',
  'numthreads': 1,
  'hide_labels': False,
  'multitask_weights': [1],
  'batchsize': 5,
  'datapath': 'd:\\research\\parlai_seq2seq\\data',
  'model': 'projects.wizard_of_wikipedia.generator.agents:EndToEndAgent',
  'model_file': '/tmp/end2end_generator/model',
  'init_model': None,
  'dict_class': None,
  'evaltask': None,
  'eval_batchsize': None,
  'display_examples': False,
  'num_epochs': -1,
  'max_train_time': -1,
  'validation_every_n_secs': -1,
  'save_every_n_secs': -1,
  'save_after_valid': False,
  'validation_every_n_epochs': -1,
  'validation_max_exs': -1,
  'short_final_eval': False,
  'validation_patience': 10,
  'validation_metric': 'accuracy',
  'validation_metric_mode': None,
  'validation_cutoff': 1.0,
  'load_from_checkpoint': False,
  'validatio

In [13]:
batch_act = SHARED['world'].batch_act(0, None)

In [15]:
batch_act[0]

{'id': 'WizTeacher',
 'text': 'Snowboarding',
 'labels': ['I love to go snowboarding on the ski slopes in the winter'],
 'chosen_topic': 'Snowboarding',
 'episode_done': False,
 'label_candidates': [],
 'knowledge': 'no_passages_used __knowledge__ no_passages_used\nSnowboarding __knowledge__ Snowboarding is a recreational activity and Olympic and Paralympic sport that involves descending a snow-covered slope while standing on a snowboard attached to a rider\'s feet.\nSnowboarding __knowledge__ The development of snowboarding was inspired by skateboarding, sledding, surfing and skiing.\nSnowboarding __knowledge__ It was developed in the United States in the 1960s, became a Winter Olympic Sport at Nagano in 1998 and first featured in the Winter Paralympics at Sochi in 2014.\nSnowboarding __knowledge__ Its popularity (as measured by equipment sales) in the United States peaked in 2007 and has been in a decline since.\nSnowboarding __knowledge__ Modern snowboarding began in 1965 when Sherm

In [7]:
SHARED['agent'].observe({"text": "夢 は 何 です か", 'episode_done': False, 'topic': "dream"})
batch = SHARED['agent'].batchify([SHARED['agent'].observation])
SHARED['agent'].model.eval()
vecs = SHARED['agent'].fixed_candidate_vecs['dream']
encs = SHARED['agent'].fixed_candidate_encs['dream']

In [17]:
batch_obs = SHARED['world'].batch_observe(1, batch_act, 0)

In [22]:
batch_obs[0]

{'id': 'WizTeacher',
 'text': 'Snowboarding',
 'labels': ['I love to go snowboarding on the ski slopes in the winter'],
 'chosen_topic': 'Snowboarding',
 'episode_done': False,
 'label_candidates': [],
 'knowledge': 'no_passages_used __knowledge__ no_passages_used\nSnowboarding __knowledge__ Snowboarding is a recreational activity and Olympic and Paralympic sport that involves descending a snow-covered slope while standing on a snowboard attached to a rider\'s feet.\nSnowboarding __knowledge__ The development of snowboarding was inspired by skateboarding, sledding, surfing and skiing.\nSnowboarding __knowledge__ It was developed in the United States in the 1960s, became a Winter Olympic Sport at Nagano in 1998 and first featured in the Winter Paralympics at Sochi in 2014.\nSnowboarding __knowledge__ Its popularity (as measured by equipment sales) in the United States peaked in 2007 and has been in a decline since.\nSnowboarding __knowledge__ Modern snowboarding began in 1965 when Sherm

In [ ]:
model_agent = world.world.get_agents()[1]

In [ ]:
dict_class = model_agent.dict

In [ ]:
v =[   1, 1869,  378,    5,    9,    2]
for t in v:
    print(dict_class.ind2bi[t])

In [ ]:
batch = model_agent.batchify(batch_obs)

In [ ]:
batch

In [ ]:
cands, cand_vecs, label_inds = model_agent._build_candidates(
            batch, source='batch', mode='train'
        )

In [ ]:
label_inds

In [ ]:
cand_vecs

In [ ]:
batch.text_vec.size(0)

In [ ]:
scores = model_agent.model(batch.text_vec, batch.bi_text_vec, cand_vecs['uni'], cand_vecs['bi'])

In [ ]:
cand_vecs

In [ ]:
x_uni = batch.text_vec
x_bi = batch.bi_text_vec
y_uni = cand_vecs['uni']
y_bi = cand_vecs['bi']

In [ ]:
x_emb, y_emb = model_agent.model.encode(x_uni, x_bi, y_uni.unsqueeze(1), y_bi.unsqueeze(1))

In [ ]:
y_emb

In [ ]:
y_emb.expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous()

In [ ]:
bsz, cand_num, seq_len = y_uni.unsqueeze(1).shape

In [ ]:
y_emb.view(bsz, cand_num, -1).expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous()

In [ ]:
y_uni_emb = model_agent.model.y_unigram_encoder(y_uni.unsqueeze(1).view(bsz*cand_num, -1))
y_bi_emb = model_agent.model.y_bigram_encoder(y_bi.unsqueeze(1).view(bsz*cand_num, -1))

In [ ]:
y_uni_emb.shape

In [ ]:
model_agent.model.C

In [ ]:
x_emb.unsqueeze(1).expand(y_emb.view(bsz, cand_num, -1).expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous().shape)

In [ ]:
y_emb = y_emb.expand(cand_num, bsz, -1).transpose(0,1).contiguous()

In [ ]:
y_emb.shape

In [1]:
from transformers import BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('/installation/~/ParlAI/data/models/bert_models/bert-wiki-ja/vocab.txt', tokenize_chinese_chars=False)

In [3]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('そっち かぁ 。 どっち に しろ あめ は ふる な だ な 。'))

[None, None, 8, None, 17, 9444, None, 11, 14045, 57, 40, 57, 8]